# Grasping with IK and Motion Planning

In this section, we’ll walk through a simple grasping task, showing how to use **inverse kinematics (IK)** together with **motion planning** to pick up a cube.

### What You Will Learn

1. Applying Inverse Kinematics (IK)
Use `franka.inverse_kinematics()` to automatically determine the robot’s joint angles for specific poses, such as hovering above or grasping the cube.

2. Using Motion Planning for Smooth Trajectories
Learn how `plan_path()` interpolates joint-space waypoints between the current and goal poses.

3. Performing a Full Grasping Sequence

In [1]:
# Suppress warning messages for clearer output
import warnings
import os

os.environ["TI_LOG_LEVEL"] = "error"
warnings.filterwarnings("ignore")

## Init and Create a Scene

Because the iGPU currently requires additional edge-case handling to ensure correct program execution when calling complex IK functions, we use the CPU backend in this lab to keep the code simple and easy to understand. If you are using an AMD dGPU, you can call the IK functions with the Vulkan backend directly, no extra handling is needed.

In [2]:
import genesis as gs
import numpy as np

########################## init ##########################
gs.init(backend=gs.cpu, theme='light')

########################## create a scene ##########################
scene = gs.Scene(
    viewer_options=gs.options.ViewerOptions(
        camera_pos=(3, -1, 1.5),
        camera_lookat=(0.0, 0.0, 0.5),
        camera_fov=30,
        max_FPS=60,
    ),
    sim_options=gs.options.SimOptions(
        dt=0.01,
    ),
    show_viewer=False,
)

[Genesis] [10:30:09] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [10:30:09] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [10:30:09] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [10:30:10] [INFO] Consider setting 'performance_mode=True' in production to maximise runtime speed, if significantly increasing compilation time is not a concern.
[Genesis] [10:30:10] [INFO] Running on [AMD RYZEN AI MAX+ 395 w/ Radeon 8060S] with backend gs.cpu. Device memory: 121.50 GB.
[Genesis] [10:30:10] [INFO] 🚀 Genesis initialized. 🔖 version: 0.3.3, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'light'.
[Genesis] [10:30:10] [INFO] Scene <b32bbaa> created.


## Add Entities and Build the Scene

Just like what we did in Lab 1, we add a plane, an arm, and a camera to the scene, and then build the scene.

In [3]:
########################## entities ##########################
plane = scene.add_entity(
    gs.morphs.Plane(),
)
cube = scene.add_entity(
    gs.morphs.Box(
        size=(0.04, 0.04, 0.04),
        pos=(0.65, 0.0, 0.02),
    )
)
franka = scene.add_entity(
    gs.morphs.MJCF(file="xml/franka_emika_panda/panda.xml"),
)
cam = scene.add_camera(
    res=(640, 480),
    pos=(3, -1, 1.5),
    lookat=(0, 0, 0.5),
    fov=30,
    GUI=True,
)
########################## build ##########################
scene.build()

[Genesis] [10:30:15] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <f5dceb5>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [10:30:15] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <f4ee1ca>, morph: <gs.morphs.Box>, material: <gs.materials.Rigid>.
[Genesis] [10:30:15] [INFO] Adding <gs.RigidEntity>. idx: 2, uid: <1728596>, morph: <gs.morphs.MJCF(file='/opt/conda/envs/py_3.12/lib/python3.12/site-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
[Genesis] [10:30:16] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [10:30:16] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias for joint `finger_joint1`.
[Genesis] [10:30:16] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [10:30:16] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to

amdgpu: os_same_file_description couldn't determine if two DRM fds reference the same file description.
If they do, bad things may happen!


## Set Control Gains

Just like Lab2, we set control gains for the arm.

In [4]:
rgb, depth, segmentation, normal = cam.render(rgb=True, depth=True, segmentation=True, normal=True)
cam.start_recording()

motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)

# set control gains
franka.set_dofs_kp(
    np.array([4500, 4500, 3500, 3500, 2000, 2000, 2000, 100, 100]),
)
franka.set_dofs_kv(
    np.array([450, 450, 350, 350, 200, 200, 200, 10, 10]),
)
franka.set_dofs_force_range(
    np.array([-87, -87, -87, -87, -12, -12, -12, -100, -100]),
    np.array([87, 87, 87, 87, 12, 12, 12, 100, 100]),
)

## Inverse Kinematics (IK)

Inverse Kinematics lets us compute the robot’s joint positions for a given end-effector pose (position + orientation). This is essential for telling the robot where the hand should go, without manually specifying every joint angle.

Let’s go step by step through the grasping process.

### 1. Define the End-Effector

In [5]:
end_effector = franka.get_link("hand")

### 2. Move to a Pre-Grasp Pose

We’ll solve IK for a position directly above the cube, and open the gripper.

In [6]:
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.25]),
    quat=np.array([0, 1, 0, 0]),
)
# gripper open pos
qpos[-2:] = 0.04

To reach this smoothly, we use the motion planner. `plan_path` interpolates between the current joint angles and the target, generating a sequence of waypoints. Executing them one by one produces a natural trajectory.

In [7]:
import logging
from tqdm import tqdm

# Set logger to warning to avoid log info.
gs.logger._logger.setLevel(logging.WARNING)

path = franka.plan_path(
    qpos_goal=qpos,
    num_waypoints=200,  # 2s duration
)
# draw the planned path
path_debug = scene.draw_debug_path(path, franka)

# execute the planned path
for waypoint in tqdm(path, desc="Executing motion path", ncols=100):
    franka.control_dofs_position(waypoint)
    cam.render()
    scene.step()

# remove the drawn path
scene.clear_debug_object(path_debug)

# allow robot to reach the last waypoint
for i in tqdm(range(100), desc="Reach the last waypoint", ncols=100):
    cam.render()
    scene.step()

Reach the last waypoint: 100%|████████████████████████████████████| 100/100 [00:02<00:00, 48.27it/s]


### 3. Lower the Gripper

Now we solve IK again, this time just above the cube’s top surface.

In [8]:
# reach
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.130]),
    quat=np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-2], motors_dof)
for i in tqdm(range(100), desc="Lower the gripper", ncols=100):
    cam.render()
    scene.step()

Lower the gripper: 100%|██████████████████████████████████████████| 100/100 [00:02<00:00, 48.09it/s]


### 4. Close the Fingers (Grasp)

We command the arm to hold its joint position, then apply closing force to the gripper fingers.

In [9]:
# grasp
franka.control_dofs_position(qpos[:-2], motors_dof)
franka.control_dofs_force(np.array([-0.5, -0.5]), fingers_dof)

for i in tqdm(range(100), desc="Close the finger", ncols=100):
    cam.render()
    scene.step()

Close the finger: 100%|███████████████████████████████████████████| 100/100 [00:02<00:00, 48.31it/s]


### 5. Lift the Cube

Finally, we solve IK for a higher position and move the arm upwards.

In [10]:
# lift
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.28]),
    quat=np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-2], motors_dof)
for i in tqdm(range(100), desc="Lift the cude", ncols=100):
    cam.render()
    scene.step()

cam.stop_recording(save_to_filename="Videos/video_03.mp4", fps=60)

Lift the cude: 100%|██████████████████████████████████████████████| 100/100 [00:02<00:00, 48.44it/s]


## Show the video

In the video, you will see the robotic arm plan a path to pick up the block. It opens its gripper and then moves downward to grasp the block.

In [11]:
from IPython.display import Video
Video(url="Videos/video_03.mp4")